# Pre Processing

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [49]:
#read raw data
raw_df = pd.read_csv("Raw Data.csv")
raw_df.head(5)

,SELLER NAME,PRODUCT NAME,PRICE,SELLER CITY,LISTING DATE,REVIEW,RATING,URL,SCRAPE DATE,SOURCE,LOCATION,Unique ID,rn,count,Quarter,URL Type,SOLD QTY,BRAND
0,MAXITOOLS,"MAXITOOLS - Stanley 7"" (180 mm) Large Angle Gr...",1224900,Kota Jakarta Barat,NaN,0.0,0.000000,https://www.lazada.co.id/products/maxitools-st...,2023-03-31,Lazada,kota jakarta barat,-i100010417.html,1,1,Q1 2023,New URL,0.0,Stanley-Black and Decker-Dewalt
1,Sriwijaya Tech,Maktec Mesin Gerinda Tangan Grinder 4inch MT90,415000,Kab. Cirebon,NaN,78.0,4.910000,https://www.lazada.co.id/products/maktec-mesin...,2022-07-31,Lazada,kab. cirebon,-i100019128.html,1,4,Q2 2022,New URL,78.0,Makita-Maktec
2,Sriwijaya Tech,Maktec Mesin Gerinda Tangan Grinder 4inch MT90,445000,Kab. Cirebon,NaN,79.0,4.900000,https://www.lazada.co.id/products/maktec-mesin...,2022-10-03,Lazada,kab. cirebon,-i100019128.html,2,4,Q3 2022,All URL,1.0,Makita-Maktec
3,Sriwijaya Tech,Maktec Mesin Gerinda Tangan Grinder 4inch MT90,409900,Kab. Cirebon,NaN,79.0,4.911392,https://www.lazada.co.id/products/maktec-mesin...,2022-12-31,Lazada,kab. cirebon,-i100019128.html,3,4,Q4 2022,All URL,0.0,Makita-Maktec
4,Sriwijaya Tech,Maktec Mesin Gerinda Tangan Grinder 4inch MT90,399900,Kab. Cirebon,NaN,79.0,4.911392,https://www.lazada.co.id/products/maktec-mesin...,2023-03-31,Lazada,kab. cirebon,-i100019128.html,4,4,Q1 2023,All URL,0.0,Makita-Maktec


In [51]:
# Show data information
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 769807 entries, 0 to 769806
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   SELLER NAME   769807 non-null  object 
 1   PRODUCT NAME  769807 non-null  object 
 2   PRICE         769807 non-null  int64  
 3   SELLER CITY   769795 non-null  object 
 4   LISTING DATE  156295 non-null  object 
 5   REVIEW        769807 non-null  float64
 6   RATING        769807 non-null  float64
 7   URL           769807 non-null  object 
 8   SCRAPE DATE   769807 non-null  object 
 9   SOURCE        769807 non-null  object 
 10  LOCATION      769795 non-null  object 
 11  Unique ID     769807 non-null  object 
 12  rn            769807 non-null  int64  
 13  count         769807 non-null  int64  
 14  Quarter       769807 non-null  object 
 15  URL Type      769807 non-null  object 
 16  SOLD QTY      769807 non-null  float64
 17  BRAND         769807 non-null  object 
dtypes: f

Judging by the columns, columns that can be used to predict the price of the tool are: Product Name, Source, Brand, and Price as it's label for training. Product name can be used because we can get more detail of the tool by it's name.

In [54]:
# Filter relevant columns
raw_df = raw_df.drop_duplicates(subset='Unique ID')
rel_col_df = raw_df[['PRODUCT NAME','BRAND','SOURCE','PRICE','Unique ID']]
rel_col_df.head(5)

,PRODUCT NAME,BRAND,SOURCE,PRICE,Unique ID
0,"MAXITOOLS - Stanley 7"" (180 mm) Large Angle Gr...",Stanley-Black and Decker-Dewalt,Lazada,1224900,-i100010417.html
1,Maktec Mesin Gerinda Tangan Grinder 4inch MT90,Makita-Maktec,Lazada,415000,-i100019128.html
5,Maktec Mesin Bor Tangan 10mm MT60 (bolak-balik),Makita-Maktec,Lazada,395000,-i100019255.html
9,Bosch GSB 120-Li - 2 Baterai Mesin Bor Obeng T...,Bosch,Lazada,1475000,-i100037639.html
13,Gergaji Mesin/ChainSaw 2.2PS - Tekiro Ryu RS5900,Tekiro Ryu,Lazada,2145000,-i100046326.html


## Define Type of Tools

In [57]:
prd_name = rel_col_df[['PRODUCT NAME']]
prd_name['PRODUCT NAME'] = prd_name['PRODUCT NAME'].str.replace(r'[^a-zA-Z]', ' ', regex=True) # remove non alphabet characters
prd_name['PRODUCT NAME'] = prd_name['PRODUCT NAME'].str.split(' ') # split the string by space

prd_name_new = prd_name.explode('PRODUCT NAME') # make each element in Product Name column into row
prd_name_new['PRODUCT NAME'] = prd_name_new['PRODUCT NAME'].str.lower()

prd_name_new_grp = prd_name_new.groupby('PRODUCT NAME').size().reset_index() # count the string occurrence
prd_name_new_grp.columns = ['Text','Size']
prd_name_new_grp['len'] = prd_name_new_grp['Text'].apply(lambda x: len(x))
prd_name_new_grp = prd_name_new_grp[(prd_name_new_grp['len']>=3)&(prd_name_new_grp['Size']>2000)] # filter string by it's length and occurrence

In [58]:
pd.set_option('display.max_rows', None) # set max number of rows to be displayed

prd_name_new_grp

,Text,Size,len
258,air,6370,3
366,alat,5260,4
445,aluminium,2018,9
509,amplas,9612,6
545,and,2782,3
572,angin,3713,5
577,angker,17642,6
597,angle,10000,5
725,arang,3991,5
778,armature,25303,8


From this list, i have to manually categorize the string

In [60]:
type_dict = {
    'amplas':'Planer','planner':'Planer','planer':'Planer','polisher':'Planer','poles':'Planer',
    'bor':'Bor','drill':'Bor',
    'gergaji':'Saw','jigsaw':'Saw','jig':'Saw','saw':'Saw',
    'palu':'Hammer','hammer':'Hammer',
    'gerinda':'Grinder','grenda':'Grinder','grinda':'Grinder','grinder':'Grinder','gurinda':'Grinder',
    'paku':'Nailer','nailer':'Nailer',
    'obeng':'Screwdriver','screw':'Screwdriver','screwdriver':'Screwdriver',
    'trimer':'Trimmer','trimmer':'Trimmer','cut':'Trimmer','cutting':'Trimmer','cutter':'Trimmer','pemotong':'Trimmer','potong':'Trimmer'
}

In [61]:
pd.set_option('display.max_rows', 20)

In [62]:
# Create function to assign the type of the tool
def type_sign(x,y):
    for key in type_dict.keys():
        if key in x:
            y = type_dict[key]
        else:
            y = y
    return y

In [63]:
rel_col_df['PRODUCT NAME lower'] = rel_col_df['PRODUCT NAME'].str.lower()
rel_col_df['Type'] = 'x'
rel_col_df['Type'] = rel_col_df.apply(lambda x:type_sign(x['PRODUCT NAME lower'],x['Type']),axis=1)

In [64]:
rel_col_df['Type'].unique()

array(['Grinder', 'Bor', 'Screwdriver', 'Saw', 'Planer', 'x', 'Trimmer',
       'Nailer', 'Hammer'], dtype=object)

## Grinder Data

We will focus on the grinder data for this project

### Filter non grinder data

In [67]:
grin_df = rel_col_df[rel_col_df['Type']=='Grinder']
grin_df.reset_index(inplace=True, drop=True)
grin_df

,PRODUCT NAME,BRAND,SOURCE,PRICE,Unique ID,PRODUCT NAME lower,Type
0,"MAXITOOLS - Stanley 7"" (180 mm) Large Angle Gr...",Stanley-Black and Decker-Dewalt,Lazada,1224900,-i100010417.html,"maxitools - stanley 7"" (180 mm) large angle gr...",Grinder
1,Maktec Mesin Gerinda Tangan Grinder 4inch MT90,Makita-Maktec,Lazada,415000,-i100019128.html,maktec mesin gerinda tangan grinder 4inch mt90,Grinder
2,Maktec Gerinda tangan Listrik 4inch MT90,Makita-Maktec,Lazada,415000,-i100100113.html,maktec gerinda tangan listrik 4inch mt90,Grinder
3,"Bosch Gerinda tangan 4"" GWS 060",Bosch,Lazada,499000,-i100141333.html,"bosch gerinda tangan 4"" gws 060",Grinder
4,Modern Gerinda Tangan 4 inch M-2350B (body Kuat),Modern,Lazada,299900,-i100181912.html,modern gerinda tangan 4 inch m-2350b (body kuat),Grinder
...,...,...,...,...,...,...,...
42404,gear gigi mesin gerinda mt90 maktec,Makita-Maktec,Bukalapak,95000,ztmluc,gear gigi mesin gerinda mt90 maktec,Grinder
42405,BROSTEL GERINDA MAKTEC MT90 - CARBON BRUSH MT ...,Makita-Maktec,Bukalapak,7000,zv4lda,brostel gerinda maktec mt90 - carbon brush mt ...,Grinder
42406,BROSTEL GERINDA MAKITA 9553 CARBON BRUSH 325 -...,Makita-Maktec,Bukalapak,6000,zv57is,brostel gerinda makita 9553 carbon brush 325 -...,Grinder
42407,"Gurinda Tangan 4"" Stanley STGS 5100",Stanley-Black and Decker-Dewalt,Bukalapak,386000,zy8f6,"gurinda tangan 4"" stanley stgs 5100",Grinder


In this project we only want to process grinder data, some sellers could be sell the product as a unit (ie.consist of multiple type of tools) so we have to filter out those data

In [74]:
# Non grinder dict
non_grin = {
    'amplas':'Planer','planner':'Planer','planer':'Planer','polisher':'Planer','poles':'Planer',
    'bor':'Bor','drill':'Bor',
    'gergaji':'Saw','jigsaw':'Saw','jig':'Saw','saw':'Saw',
    'palu':'Hammer','hammer':'Hammer',
    'paku':'Nailer','nailer':'Nailer',
    'obeng':'Screwdriver','screw':'Screwdriver','screwdriver':'Screwdriver',
    'trimer':'Trimmer','trimmer':'Trimmer','cut':'Trimmer','cutting':'Trimmer','cutter':'Trimmer','pemotong':'Trimmer','potong':'Trimmer'}

def type_sign_grin(x,y):
    for key in non_grin.keys():
        if key in x:
            y = type_dict[key]
        else:
            y = y
    return y


grin_df['Type'] = 'Grinder' # At first we set everything to grinder
grin_df['Type'] = grin_df.apply(lambda x:type_sign_grin(x['PRODUCT NAME lower'],x['Type']),axis=1) # then we assign again type but without grinder

grin_df= grin_df[grin_df['Type']=='Grinder'] # we can filter out the non grinder type

In [75]:
# Show descriptive statistic of the data
grin_df.describe()

,PRICE
count,3.893900e+04
mean,5.254827e+05
std,8.369697e+05
min,3.000000e+02
25%,1.069000e+05
50%,3.720000e+05
75%,5.980000e+05
max,5.158100e+07


As we can see the data has wide range of variance, but we can assume that grinder has to has price more then IDR 100K

In [77]:
grin_df = grin_df[grin_df['PRICE']>=1e5]
grin_df.reset_index(inplace=True, drop=True)
grin_df

,PRODUCT NAME,BRAND,SOURCE,PRICE,Unique ID,PRODUCT NAME lower,Type
0,"MAXITOOLS - Stanley 7"" (180 mm) Large Angle Gr...",Stanley-Black and Decker-Dewalt,Lazada,1224900,-i100010417.html,"maxitools - stanley 7"" (180 mm) large angle gr...",Grinder
1,Maktec Mesin Gerinda Tangan Grinder 4inch MT90,Makita-Maktec,Lazada,415000,-i100019128.html,maktec mesin gerinda tangan grinder 4inch mt90,Grinder
2,Maktec Gerinda tangan Listrik 4inch MT90,Makita-Maktec,Lazada,415000,-i100100113.html,maktec gerinda tangan listrik 4inch mt90,Grinder
3,"Bosch Gerinda tangan 4"" GWS 060",Bosch,Lazada,499000,-i100141333.html,"bosch gerinda tangan 4"" gws 060",Grinder
4,Modern Gerinda Tangan 4 inch M-2350B (body Kuat),Modern,Lazada,299900,-i100181912.html,modern gerinda tangan 4 inch m-2350b (body kuat),Grinder
...,...,...,...,...,...,...,...
29839,MAKTEC MT963 MESIN GERINDA GURINDA GRENDA TANG...,Makita-Maktec,Bukalapak,1000000,zt7eyu,maktec mt963 mesin gerinda gurinda grenda tang...,Grinder
29840,MESIN GURINDA MAKITA 7 INCHI GA 7020,Makita-Maktec,Bukalapak,2070000,ztao5v,mesin gurinda makita 7 inchi ga 7020,Grinder
29841,MAKTEC MT90 MESIN GERINDA GURINDA GRENDA TANGA...,Makita-Maktec,Bukalapak,425000,ztema9,maktec mt90 mesin gerinda gurinda grenda tanga...,Grinder
29842,"Gurinda Tangan 4"" Stanley STGS 5100",Stanley-Black and Decker-Dewalt,Bukalapak,386000,zy8f6,"gurinda tangan 4"" stanley stgs 5100",Grinder


### Filter sparepart data

In this data, seller could be sell only sparepart (ie part of the grinder for repair) not a whole grinder machine, so we want to filter out those data

Now we will try to get the detail of the grinder from the product name, in this case is the grinder size

In [86]:
grin_df = grin_df[~(grin_df['PRODUCT NAME lower'].str.contains(r'(switch|saklar|arma|angk|spare|dust)',regex=True))]

### Grinder specification

One of the aspect that can be used to determine the price of grinder is specification of the grinder. Grinder can have many specifications, but for this project we only use 3 specifications.

- Cordless : whether the grinder using battery or not
- Auto : whether the grinder is automatic or not
- Size : the size of the grinder

We can try to find these specifications from the product name column

In [89]:
grin_df['Cordless'] = grin_df['PRODUCT NAME lower'].str.contains('cord')
grin_df['Cordless'].value_counts()

Cordless
False    23502
True      1045
Name: count, dtype: int64

In [91]:
grin_df['Auto'] = grin_df['PRODUCT NAME lower'].str.contains('auto')
grin_df['Auto'].value_counts()

Auto
False    24544
True         3
Name: count, dtype: int64

In [93]:
# Mapping auto and cordless column
grin_df['Auto'] = grin_df['Auto'].map({True:1,False:0})
grin_df['Cordless'] = grin_df['Cordless'].map({True:1,False:0})

In [95]:
# Extract size of the grinder
grin_df['PRODUCT NAME lower'] = grin_df['PRODUCT NAME lower'].str.replace(r' in', 'in', regex=True)
grin_df['PRODUCT NAME lower'] = grin_df['PRODUCT NAME lower'].str.split(' ')

grin_df = grin_df.explode('PRODUCT NAME lower')

grin_df['Check True'] = grin_df['PRODUCT NAME lower'].apply(lambda x: True if '"' in x or 'inc' in x else False) # we use inch as unit of measurement
grin_df = grin_df[grin_df['Check True']==True]

grin_df['Size'] = grin_df['PRODUCT NAME lower'].str.findall(r'\d+') # finding all numeric character

grin_df = grin_df[grin_df['Size'].isna()==False]

grin_df = grin_df.explode('Size')
grin_df['Size'] = grin_df['Size'].astype('float')
grin_df = grin_df[(grin_df['Size']<10)&(grin_df['Size']>3)] # Grinder usually has size ranging from 4 inch to 9 inch

In [97]:
grin_df['Size'].unique()

array([7., 4., 5., 6., 9., 8.])

### Spliting brand name

In brand name column, there are values which are group of brand (ie consist of many brands) not individual brand, we will split those brand group into individual brand

In [100]:
grin_df['Brand True'] = grin_df['BRAND'].str.split(r"(-| and )")
grin_df = grin_df.explode('Brand True')

grin_df = grin_df[~(grin_df['Brand True'].isin(['-',' and ','Black']))]

grin_df['PRODUCT NAME lower'] = grin_df['PRODUCT NAME'].str.lower()

def check(x,y):
    if x in y :
        return True
    else:
        return False
    
grin_df['Brand Check'] = grin_df.apply(lambda x:check(x['Brand True'].lower(),x['PRODUCT NAME lower']),axis=1)

grin_df = grin_df[grin_df['Brand Check']==True]

grin_df['BRAND'] = grin_df['Brand True']

In [102]:
# Lets tidy up the columns that will be used
grin_df = grin_df[['BRAND','SOURCE','Size','PRICE','Cordless','Auto']]
grin_df.columns =['Brand','Source','Size','Price','Cordless','Auto']
grin_df

,Brand,Source,Size,Price,Cordless,Auto
0,Stanley,Lazada,7.0,1224900,0,0
0,Stanley,Lazada,7.0,1224900,0,0
1,Maktec,Lazada,4.0,415000,0,0
2,Maktec,Lazada,4.0,415000,0,0
3,Bosch,Lazada,4.0,499000,0,0
...,...,...,...,...,...,...
29840,Makita,Bukalapak,7.0,2070000,0,0
29841,Makita,Bukalapak,4.0,425000,0,0
29841,Maktec,Bukalapak,4.0,425000,0,0
29842,Stanley,Bukalapak,4.0,386000,0,0


In [104]:
grin_df.drop_duplicates(inplace=True, ignore_index=True)
grin_df

,Brand,Source,Size,Price,Cordless,Auto
0,Stanley,Lazada,7.0,1224900,0,0
1,Maktec,Lazada,4.0,415000,0,0
2,Bosch,Lazada,4.0,499000,0,0
3,Modern,Lazada,4.0,299900,0,0
4,Dewalt,Lazada,7.0,1988000,0,0
...,...,...,...,...,...,...
7012,Dongcheng,Bukalapak,4.0,357500,0,0
7013,Makita,Bukalapak,4.0,690000,0,0
7014,Makita,Bukalapak,7.0,2225000,0,0
7015,Makita,Bukalapak,7.0,2070000,0,0


In [106]:
grin_df['Brand'].value_counts()

Brand
Makita        1356
Maktec        1309
Dck            957
Bosch          573
Dongcheng      538
Modern         467
Tekiro Ryu     423
Dewalt         408
Dca            393
Ingco          302
Stanley        198
Decker          93
Name: count, dtype: int64

### Outlier

Now, we will dealing the outlier of the data. We will dealing outlier for each size

In [109]:
# cordless or auto grinder
crd_aut = grin_df[(grin_df['Auto']==1)|(grin_df['Cordless']==1)]
crd_aut

,Brand,Source,Size,Price,Cordless,Auto
15,Ingco,Lazada,4.0,845000,1,0
112,Ingco,Lazada,4.0,892525,1,0
113,Ingco,Lazada,4.0,1406525,1,0
114,Ingco,Lazada,4.0,1494273,1,0
124,Tekiro Ryu,Lazada,4.0,1099300,1,0
...,...,...,...,...,...,...
6824,Dca,Tokopedia,4.0,2076000,1,0
6845,Makita,Tokopedia,4.0,11739100,1,0
6849,Dck,Tokopedia,4.0,548880,0,1
6876,Bosch,Tokopedia,5.0,1794900,1,0


In [113]:
# we will use inter quartile range to define the outliers
size = grin_df['Size'].unique()
new_grin_df = pd.DataFrame()
for sz in size:
    temp_df = grin_df[(grin_df['Size']==sz)&(grin_df['Cordless']==0)&(grin_df['Auto']==0)]
    iqr = temp_df['Price'].quantile(0.75) - temp_df['Price'].quantile(0.25)
    lower_bound = temp_df['Price'].quantile(0.25) - 1.5*iqr
    upper_bound = temp_df['Price'].quantile(0.75) + 1.5*iqr
    temp_df = temp_df[(temp_df['Price']>=lower_bound)&(temp_df['Price']<=upper_bound)]
    new_grin_df = pd.concat([new_grin_df,temp_df])

new_grin_df = pd.concat([new_grin_df,crd_aut])

In [115]:
grin_df = new_grin_df.copy()

In [117]:
# Export grinder data to csv for further process
grin_df.to_csv('Grinder Pre-Processed.csv',index=False)